Version1, VGGT with refiner

Version2, VGGT w/o refiner

Version3, LightGlue with refiner

In [1]:
DEBUG = False

import os
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    DEBUG = False

SAVE_FLAG = (not os.getenv('KAGGLE_IS_COMPETITION_RERUN') and not DEBUG)

In [2]:
def install():

    !dpkg -i --force-depends /kaggle/input/colmap-offline-installer/archives/*.deb >/dev/null 2>&1
    
    !cp -r /kaggle/input/incremental-model-refiner-binary/colmap-bin /kaggle/working
    !chmod +x /kaggle/working/colmap-bin/bin/colmap

    import os
    os.environ["COLMAP_PATH"] = "/kaggle/working/colmap-bin/bin/colmap"
    
    COLMAP_PATH = os.environ.get("COLMAP_PATH", "colmap")
    !{COLMAP_PATH} incremental_model_refiner -h

In [3]:
def preprocess():
    
    !mkdir -p /root/.cache/torch/hub/checkpoints

    # KeyNet - adalam
    !cp -v /kaggle/input/kornia-local-feature-weights/* /root/.cache/torch/hub/checkpoints/

    # VGGT Tracker
    !cp -v /kaggle/input/lightglue-offline/ALIKED/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
    !cp -v /kaggle/input/lightglue-offline/ALIKED/aliked_lightglue_v0-1_arxiv.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth
    !cp -v /kaggle/input/lightglue-offline/SuperPoint/superpoint_v1.pth /root/.cache/torch/hub/checkpoints/

    # ALIKED - LightGlue
    !cp -v /kaggle/input/lightglue-offline/ALIKED/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
    !cp -v /kaggle/input/lightglue-offline/ALIKED/aliked_lightglue_v0-1_arxiv.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv.pth
    
    return

In [4]:
def inference():

    !python3 /kaggle/input/imc2025-exp66-vggt-with-incremental-model-refiner/inference.py \
        param=submission

    return

In [5]:
import pandas as pd
import numpy as np

def postprocess():
    
    df = pd.read_csv("/kaggle/working/submission.csv")
    !rm /kaggle/working/*.csv
    !rm -r /kaggle/working/outputs
    !rm -r /kaggle/working/featureout
    !rm -r /kaggle/working/colmap-bin

    df.to_csv("/kaggle/working/submission.csv", index=False)

In [6]:
if SAVE_FLAG:
    !cp /kaggle/input/image-matching-challenge-2025/sample_submission.csv /kaggle/working/submission.csv
else:
    install()
    preprocess()
    inference()
    postprocess()